In [1]:
# eyeball and iris tracking (old way)
import numpy as np
import cv2
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob

def sequential_label(binary_in):
    a = binary_in.shape
    a = np.array([int(v) for v in a])
    I = binary_in
    label = 0
    labeled_image = np.zeros((a[0],a[1]))
    record = []
    for i in range(1,a[0]):
        for j in range(1,a[1]):
            if I[i][j]!=0: 
                if labeled_image[i-1][j-1]!=0:
                    labeled_image[i][j] = labeled_image[i-1][j-1]

                elif labeled_image[i][j-1]==0 and labeled_image[i-1][j]==0:
                    label = label+1
                    labeled_image[i][j] = label
                elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]==0:
                    labeled_image[i][j] = labeled_image[i-1][j]

                elif labeled_image[i-1][j]==0 and labeled_image[i][j-1]!=0:
                    labeled_image[i][j] = labeled_image[i][j-1]    

                elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]!=0:

                    if labeled_image[i][j-1]==labeled_image[i-1][j]:
                        labeled_image[i][j] = labeled_image[i][j-1]

                    else: 
                        minValue = min(labeled_image[i][j-1],labeled_image[i-1][j])
                        maxValue = max(labeled_image[i][j-1],labeled_image[i-1][j])
                        labeled_image[i][j] = labeled_image[i][j-1]
                        # record equivalence of labels
                        labeled_image = np.where(labeled_image == minValue, maxValue, labeled_image)

    labelled_img = labeled_image
    # normalized
    
    max_labelled_img = labelled_img.max()
    labelled_img = (labelled_img*(255/max_labelled_img)).astype('uint8')
    
        
    return labelled_img

def moment_opencv(labelled_in):
    res = {}
    for label_idx in np.unique(labelled_in):
        if label_idx == 0:
            continue
        masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
        moments = cv2.moments(masked_in)
        res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                          moments['m02'],moments['m11'],moments['m20'],
                          moments['mu02'],moments['mu11'],moments['mu20']]
    return res

def binarize(gray_in, threshold=128):
    #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
    binary_image = np.where(gray_in>threshold,0,255).astype('uint8')
    return binary_image

def find_iris(eye_image):
    eye = np.array(eye_image)
    gray_eye = np.array(eye)
    [H,L] = eye_image.shape    
    scaler_x= H/40.
    scaler_y=L/40.
    gray_eye = cv2.resize(gray_eye,(40,40))
    #cv2.imshow('frame2',gray_eye)

    print (scaler_x,scaler_y)
    gray_eye = cv2.medianBlur(gray_eye, 5)
    gray_eye = binarize(gray_eye,50)
    label = sequential_label(gray_eye)
    cv2.imshow('frame3',label)
    if label.max() == 0:
        x_hat = 0
        y_hat = 0
        
    else:
        
        res = moment_opencv(label)
        list_A = []
        list_i = []
        count = 0
        for i in res:
            list_A.append(res[i][0])
            count = count+1
            list_i.append(i)
        index = np.where(list_A == max(list_A))
                
        index = np.array(index)       
        
    
        if index.shape == (1,0):
            index = 0
            
        ii = list_i[index]

        x_hat = round((float(res[ii][1])/float(res[ii][0]))*scaler_x)
        y_hat = round((float(res[ii][2])/float(res[ii][0]))*scaler_y)
        r = 5
        return x_hat,y_hat

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
cap = cv2.VideoCapture(0)
count = 0
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,10)
        print (eyes)
        count  = 0 
        for (ex,ey,ew,eh) in eyes:
#           ex_smallest = np.minimum(ex)
#           index_smallest = where(ex==ex_smallest)
            count_eye = count_eye +1
            if count_eye == 2:
                break
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            eye_img = roi_gray[ey:ey+eh,ex:ex+ew]
            #eye_color = roi_color[ey:ey+eh,ex:ex+ew]
            count= count +1
            eye_image = eye_img
            if eye_image is not None:
                try:
                    x_hat,y_hat = find_iris(eye_image)

                except:
                    print ('cant find iris!')
                else:                
                    r = 2
                    cv2.circle(roi_color, (np.int64(ex+x_hat), np.int64(ey+y_hat)), r, (0, 255, 0), 2)

                cv2.imshow('frame2',eye_image)
            
            
    cv2.imshow('frame1',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


()
()
()
()
()
()
[[141  54  27  27]]


NameError: name 'count_eye' is not defined

In [5]:
import Matrix_CV_ML3D as DImage
tests = DImage.Matrix_CV_ML3D("train_data/buffer",25,50)
tests.build_ML_matrix()

ValueError: invalid literal for int() with base 10: 'image'

In [1]:
# eyeball and using ML (move it)
import numpy as np
import cv2
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math
# haarcascade_mcs_upperbody
model_left_eye = load_model('left_eye_model.h5')
model_right_eye = load_model('right_eye_model.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
cap = cv2.VideoCapture(0)
count = 0
eyecount = 0
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,10)
        if eyes is None:
            pass
        num_eyes = np.size(eyes)
        eyecount = eyecount+1
        num_eyes = num_eyes/4
        if num_eyes == 1:
            cv2.putText(frame,"stop!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
            
        if num_eyes == 2:
            # we have to find two eyes 
            count = 0
            if eyes[0,0]<eyes[1,0]:
                check = 1
            else:
                check = 2
            for (ex,ey,ew,eh) in eyes:
                count = count+1
                if count == check:
                    # left eyes
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    eye_img = roi_gray[ey:ey+eh,ex:ex+ew]
                    #eye_color = roi_color[ey:ey+eh,ex:ex+ew]
                    eye_image = eye_img
                    cv2.imshow('left eye',eye_image)
                    resized_eyes = cv2.resize(eye_image, (50, 50))
                    cut_image = resized_eyes[10:35,:]
                    cut_image_expend = np.zeros((25,50,3))
                    cut_image_expend[:,:,0] = cut_image
                    cut_image_expend[:,:,1] = cut_image
                    cut_image_expend[:,:,2] = cut_image
                    img = np.reshape(cut_image_expend,(1,3,50,25))
                    pred  = model_left_eye.predict(img)
                    predicted = np.argmax(pred,axis=1)
                    left_eye_result = predicted
                else:
                    # right eyes
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    eye_img = roi_gray[ey:ey+eh,ex:ex+ew]
                    #eye_color = roi_color[ey:ey+eh,ex:ex+ew]
                    eye_image = eye_img
                    cv2.imshow('right eye',eye_image)
                    resized_eyes = cv2.resize(eye_image, (50, 50))
                    cut_image = resized_eyes[10:35,:]
                    cut_image_expend = np.zeros((25,50,3))
                    cut_image_expend[:,:,0] = cut_image
                    cut_image_expend[:,:,1] = cut_image
                    cut_image_expend[:,:,2] = cut_image
                    img = np.reshape(cut_image_expend,(1,3,50,25))
                    pred  = model_right_eye.predict(img)
                    predicted = np.argmax(pred,axis=1)
                    right_eye_result = predicted
                    
            print (left_eye_result,right_eye_result)
            if left_eye_result != right_eye_result:
                    print ('move forward!')
                    cv2.putText(frame,"move forward!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
            if left_eye_result == right_eye_result:
                if left_eye_result == 0:
                    print ('move forward!')
                    cv2.putText(frame,"move forward!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                elif left_eye_result == 1:
                    print ('move forward!')
                    cv2.putText(frame,"move forward!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                elif left_eye_result == 2:
                    print ('move left!')
                    cv2.putText(frame,"move left!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                elif left_eye_result == 3:
                    print ('move right!')
                    cv2.putText(frame,"move right!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    cv2.imshow('frame1',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


Using TensorFlow backend.


error: C:\ci\opencv_1512688052760\work\modules\objdetect\src\cascadedetect.cpp:1698: error: (-215) !empty() in function cv::CascadeClassifier::detectMultiScale


In [ ]:
# eyeball and using number of eyes (move it)
import numpy as np
import cv2
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math

In [11]:
model_left_eye = load_model('left_eye_model.h5')
model_right_eye = load_model('right_eye_model.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
cap = cv2.VideoCapture(0)
count = 0
eyecount = 0
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    if np.size(faces)==0:
    
        print ('stop')
        cv2.putText(frame,"stop", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)

    for (x,y,w,h) in faces:
        ##########
        num_face = np.size(faces)
        if np.size(faces)>1:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,10)
        if np.size(eyes)==0:
            print ('stop')
            cv2.putText(frame,"stop", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        num_eyes = np.size(eyes)
        eyecount = eyecount+1
        num_eyes = num_eyes/4
        if num_eyes == 1:
            if eyes[0,0]<faces[0,2]/2:
                print ('move right!')
                cv2.putText(frame,"move right!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
            else:
                print ('move left!')
                cv2.putText(frame,"move left!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        if num_eyes == 2:
            print ('move forward!')
            cv2.putText(frame,"move forward!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    cv2.imshow('frame1',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

stop
stop
move left!
move left!
move forward!
move left!
stop
stop
stop
move right!
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
move right!
move forward!
move forward!
move forward!
move right!
move right!
move forward!
stop
stop
move forward!
stop
stop
stop
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
stop
stop
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
move forward!
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
sto

stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop


In [3]:
# collect eyes data
import numpy as np
import cv2
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math
model_left_eye = load_model('left_eye_model.h5')
model_right_eye = load_model('right_eye_model.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
cap = cv2.VideoCapture(0)
eyecount = 0
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,10)
        if eyes is None:
            pass
        num_eyes = np.size(eyes)
        eyecount = eyecount+1
        num_eyes = num_eyes/4
        if num_eyes == 2:
            # we have to find two eyes 
            count = 0
            if eyes[0,0]<eyes[1,0]:
                check = 1
            else:
                check = 2
            left_eye_result = 0
            right_eye_result = 0
            for (ex,ey,ew,eh) in eyes:
                count = count+1
                if count == check:
                    # left eyes
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    eye_img = roi_gray[ey:ey+eh,ex:ex+ew]
                    #eye_color = roi_color[ey:ey+eh,ex:ex+ew]
                    eye_image = eye_img
                    cv2.imshow('left eye',eye_image)
                    resized_eyes = cv2.resize(eye_image, (50, 50))
                    cut_image = resized_eyes[10:35,:]
                    dir  = "train_data/Jinxin_left_eye/left_eye_%d.jpg" % count
                    cv2.imwrite(dir,cut_image)

                else:
                    # right eyes
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                    eye_img = roi_gray[ey:ey+eh,ex:ex+ew]
                    #eye_color = roi_color[ey:ey+eh,ex:ex+ew]
                    eye_image = eye_img
                    cv2.imshow('right eye',eye_image)
                    resized_eyes = cv2.resize(eye_image, (50, 50))
                    cut_image = resized_eyes[10:35,:]
                    dir  = "train_data/Jinxin_right_eye/right_eye_%d.jpg" % count
                    cv2.imwrite(dir,cut_image)
                
                
    cv2.imshow('frame1',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Using TensorFlow backend.


In [2]:
import cv2
img= cv2.resize(cut_image,(50,25), interpolation = cv2.INTER_CUBIC)
img.shape

cv2.imshow('figure',img)

NameError: name 'cut_image' is not defined

In [2]:
# train the model left eye model 
# 0 look straight 
# 1 move forward
# 2 move left 
# 3 move right 

# Train CNN model
import Matrix_CV_ML3D as DImage
import Matrix_CV_ML as NDImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math

K.set_image_dim_ordering('th')
x = DImage.Matrix_CV_ML("train_data/Jinxin_left_eyes",25,50)
x.build_ML_matrix()

labels = np.zeros(288)
labels[0:20] = 0
labels[21:57] = 1
labels[58:58+110] = 2
labels[169:287] = 3
x.labels = labels
y = np_utils.to_categorical(x.labels)
x = x.global_matrix
x = x.astype('float32')/255 # wrong! should choose max in image 

model = Sequential()

model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(3,50,25)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
   
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 9. Fit model on training data
model.fit(x, y, batch_size=32, nb_epoch=80,  shuffle = True, verbose=2)
 
# 10. Evaluate model on test data
score = model.evaluate(x, y, verbose=0)

# Save Training Model

model.save('left_eye_model.h5')  # creates a HDF5 file 'my_model.h5'

C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(3, 50, 25...)`
C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:56: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (288, 1250)

In [2]:

# Test left eye CNN model
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math
test = DImage.Matrix_CV_ML3D("test_data/Jinxin_left_eyes_test",25,50)
test.build_ML_matrix()
test = test.global_matrix
test = test.astype('float32')/255
model = load_model('left_eye_model.h5')
pred  = model.predict(test)
predicted = np.argmax(pred,axis=1)
print (predicted)

(1, 3, 50, 25)(1, 3, 50, 25)
(1, 3, 50, 25)(2, 3, 50, 25)
(1, 3, 50, 25)(3, 3, 50, 25)
(1, 3, 50, 25)(4, 3, 50, 25)
(1, 3, 50, 25)(5, 3, 50, 25)
(1, 3, 50, 25)(6, 3, 50, 25)
(1, 3, 50, 25)(7, 3, 50, 25)
(1, 3, 50, 25)(8, 3, 50, 25)
(1, 3, 50, 25)(9, 3, 50, 25)
(1, 3, 50, 25)(10, 3, 50, 25)
(1, 3, 50, 25)(11, 3, 50, 25)
(1, 3, 50, 25)(12, 3, 50, 25)
(1, 3, 50, 25)(13, 3, 50, 25)
(1, 3, 50, 25)(14, 3, 50, 25)
(1, 3, 50, 25)(15, 3, 50, 25)
(1, 3, 50, 25)(16, 3, 50, 25)
(1, 3, 50, 25)(17, 3, 50, 25)
(1, 3, 50, 25)(18, 3, 50, 25)
(1, 3, 50, 25)(19, 3, 50, 25)
(1, 3, 50, 25)(20, 3, 50, 25)
(1, 3, 50, 25)(21, 3, 50, 25)
(1, 3, 50, 25)(22, 3, 50, 25)
[0 0 0 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3]


In [6]:
# train the model right eye model 
# 0 look straight 
# 1 move forward
# 2 move left 
# 3 move right 

# Train CNN model
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math

K.set_image_dim_ordering('th')
x = DImage.Matrix_CV_ML3D("train_data/Jinxin_right_eyes",25,50)
x.build_ML_matrix()

labels = np.zeros(316)
labels[0:23] = 0
labels[24:60] = 1
labels[61:61+110] = 2
labels[172:315] = 3
x.labels = labels
y = np_utils.to_categorical(x.labels)
x = x.global_matrix
x = x.astype('float32')/255

model = Sequential()

model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(3,50,25)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
   
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 9. Fit model on training data
model.fit(x, y, batch_size=32, nb_epoch=80,  shuffle = True, verbose=2)

# 10. Evaluate model on test data
score = model.evaluate(x, y, verbose=0)

# Save Training Model

model.save('right_eye_model.h5')  # creates a HDF5 file 'my_model.h5'

(1, 3, 50, 25)(1, 3, 50, 25)
(1, 3, 50, 25)(2, 3, 50, 25)
(1, 3, 50, 25)(3, 3, 50, 25)
(1, 3, 50, 25)(4, 3, 50, 25)
(1, 3, 50, 25)(5, 3, 50, 25)
(1, 3, 50, 25)(6, 3, 50, 25)
(1, 3, 50, 25)(7, 3, 50, 25)
(1, 3, 50, 25)(8, 3, 50, 25)
(1, 3, 50, 25)(9, 3, 50, 25)
(1, 3, 50, 25)(10, 3, 50, 25)
(1, 3, 50, 25)(11, 3, 50, 25)
(1, 3, 50, 25)(12, 3, 50, 25)
(1, 3, 50, 25)(13, 3, 50, 25)
(1, 3, 50, 25)(14, 3, 50, 25)
(1, 3, 50, 25)(15, 3, 50, 25)
(1, 3, 50, 25)(16, 3, 50, 25)
(1, 3, 50, 25)(17, 3, 50, 25)
(1, 3, 50, 25)(18, 3, 50, 25)
(1, 3, 50, 25)(19, 3, 50, 25)
(1, 3, 50, 25)(20, 3, 50, 25)
(1, 3, 50, 25)(21, 3, 50, 25)
(1, 3, 50, 25)(22, 3, 50, 25)
(1, 3, 50, 25)(23, 3, 50, 25)
(1, 3, 50, 25)(24, 3, 50, 25)
(1, 3, 50, 25)(25, 3, 50, 25)
(1, 3, 50, 25)(26, 3, 50, 25)
(1, 3, 50, 25)(27, 3, 50, 25)
(1, 3, 50, 25)(28, 3, 50, 25)
(1, 3, 50, 25)(29, 3, 50, 25)
(1, 3, 50, 25)(30, 3, 50, 25)
(1, 3, 50, 25)(31, 3, 50, 25)
(1, 3, 50, 25)(32, 3, 50, 25)
(1, 3, 50, 25)(33, 3, 50, 25)
(1, 3, 50, 25)(34, 

C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(3, 50, 25...)`
C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:55: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/80
 - 4s - loss: 1.3564 - acc: 0.3892
Epoch 2/80
 - 3s - loss: 1.1431 - acc: 0.4430
Epoch 3/80
 - 3s - loss: 1.0195 - acc: 0.5063
Epoch 4/80
 - 3s - loss: 0.9061 - acc: 0.6171
Epoch 5/80
 - 3s - loss: 0.7665 - acc: 0.7184
Epoch 6/80
 - 3s - loss: 0.7287 - acc: 0.6835
Epoch 7/80
 - 3s - loss: 0.6592 - acc: 0.7057
Epoch 8/80
 - 3s - loss: 0.5961 - acc: 0.7595
Epoch 9/80
 - 3s - loss: 0.5920 - acc: 0.7468
Epoch 10/80
 - 3s - loss: 0.5565 - acc: 0.7563
Epoch 11/80
 - 3s - loss: 0.5336 - acc: 0.7848
Epoch 12/80
 - 3s - loss: 0.5312 - acc: 0.7595
Epoch 13/80
 - 3s - loss: 0.5263 - acc: 0.7816
Epoch 14/80
 - 3s - loss: 0.4912 - acc: 0.8070
Epoch 15/80
 - 3s - loss: 0.4824 - acc: 0.7816
Epoch 16/80
 - 3s - loss: 0.4946 - acc: 0.7848
Epoch 17/80
 - 3s - loss: 0.4946 - acc: 0.8070
Epoch 18/80
 - 3s - loss: 0.4702 - acc: 0.8101
Epoch 19/80
 - 3s - loss: 0.4698 - acc: 0.8070
Epoch 20/80
 - 3s - loss: 0.4547 - acc: 0.8259
Epoch 21/80
 - 3s - loss: 0.4475 - acc: 0.8070
Epoch 22/80
 - 3s - lo

In [7]:
# Test left eye CNN model
import Matrix_CV_ML3D as DImage
#import Matrix_CV_ML_realtime as RImage
from keras import backend as K
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
import cv2
import numpy as np
import math
test = DImage.Matrix_CV_ML3D("test_data/Jinxin_right_eyes_test",25,50)
test.build_ML_matrix()
test = test.global_matrix
test = test.astype('float32')/255
model = load_model('right_eye_model.h5')
pred  = model.predict(test)
predicted = np.argmax(pred,axis=1)
print (predicted)

(1, 3, 50, 25)(1, 3, 50, 25)
(1, 3, 50, 25)(2, 3, 50, 25)
(1, 3, 50, 25)(3, 3, 50, 25)
(1, 3, 50, 25)(4, 3, 50, 25)
(1, 3, 50, 25)(5, 3, 50, 25)
(1, 3, 50, 25)(6, 3, 50, 25)
(1, 3, 50, 25)(7, 3, 50, 25)
(1, 3, 50, 25)(8, 3, 50, 25)
(1, 3, 50, 25)(9, 3, 50, 25)
(1, 3, 50, 25)(10, 3, 50, 25)
(1, 3, 50, 25)(11, 3, 50, 25)
(1, 3, 50, 25)(12, 3, 50, 25)
(1, 3, 50, 25)(13, 3, 50, 25)
(1, 3, 50, 25)(14, 3, 50, 25)
(1, 3, 50, 25)(15, 3, 50, 25)
(1, 3, 50, 25)(16, 3, 50, 25)
(1, 3, 50, 25)(17, 3, 50, 25)
(1, 3, 50, 25)(18, 3, 50, 25)
(1, 3, 50, 25)(19, 3, 50, 25)
(1, 3, 50, 25)(20, 3, 50, 25)
(1, 3, 50, 25)(21, 3, 50, 25)
(1, 3, 50, 25)(22, 3, 50, 25)
(1, 3, 50, 25)(23, 3, 50, 25)
(1, 3, 50, 25)(24, 3, 50, 25)
(1, 3, 50, 25)(25, 3, 50, 25)
[0 0 0 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 2 3 3]
